In [12]:
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import title
from random import randrange
sns.set(rc={'figure.figsize':(15,8)})

In [13]:
df = pd.read_csv("./data/abalone.data", names=["sex", "length", "diameter", "height", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "rings"])

In [14]:
df

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [15]:
df.describe()

,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [16]:
pd_df['difficulty'] = 'Unknown'
pd_df.loc[pd_df['Time'].between(0, 30, inclusive=False), 'difficulty'] = 'Easy'
pd_df.loc[pd_df['Time'].between(30, 60, inclusive=True), 'difficulty'] = 'Medium'



NameError: name 'pd_df' is not defined

In [26]:
df.loc[df['rings'].between(0, 7, inclusive='both'), 'ring_class'] = int(1)
df.loc[df['rings'].between(8, 10, inclusive='both'), 'ring_class'] = int(2)
df.loc[df['rings'].between(11, 15, inclusive='both'), 'ring_class'] = int(3)
df.loc[df['rings'] > 15, 'ring_class'] = int(4)


In [27]:
df

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings,ring_class
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15,3.0
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7,1.0
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9,2.0
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10,2.0
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7,1.0
...,...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11,3.0
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10,2.0
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9,2.0
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10,2.0


In [28]:
df['ring_class'].value_counts()

2.0    1891
3.0    1186
1.0     839
4.0     261
Name: ring_class, dtype: int64